For reading the dataset 

In [6]:
import pandas as pd
import numpy as np
from copy import deepcopy
from distutils.command.build_scripts import first_line_re
from tkinter.tix import COLUMN
# Import deque for the stack structure, copy for deep copy nodes
from collections import deque


class Dataset:
    def __init__(self,df):
        self.df=df
        self.x = self.df.iloc[:,:-1]
        self.y = self.df.iloc[:,-1]
        # self.y = self.df.iloc[:-1]

    @classmethod
    def constructFromFile(cls,filePath):
        df = pd.read_csv(filePath,header=None)
        df.columns = [f"feature{i+1}" for i in range(len(df.columns))]
        df.rename(columns = {f'feature{len(df.columns)}':'class'}, inplace = True)
        return cls(df) 

class Sonar(Dataset):
    def __init__(self,df):
        super().__init__(df)
    
class Wbcd(Dataset):
    def __init__(self,df):
        super().__init__(df)

sonar  = Sonar.constructFromFile("./sonar/sonar.data")
wbcd = Wbcd.constructFromFile("./wbcd/wbcd.data")
wbcd.df


,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,class
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,2
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,2
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,2
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,2
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,2
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,2
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,2
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,2


> Use **Creator** to define the type of individuals and fitness classes.

In [ ]:
from asyncio import constants
from json import tool
from deap import creator, base, gp, tools, algorithms # core functionality of DEAP
import array
import random
import json
import math # for checking the fitness of an individual, i.e. math.isinf(weight)
import matplotlib.pyplot as plt
# Python internal operators for object comparisons, 
# logical comparisons, arithmetic operations, and sequence operations
import operator 


# creator is  usually used to define the type of the individual and fitness classes

# goal:to maximize the value and do not exceed the capacity of the knapsack
# define strategies with different priorities for optimizing multiple goals by using FitnessCompound
# 1 for maximize value, -1 for minimize weight, 
# creator.create("FitnessCompound", base.Fitness, weights=(1.0,-1.0)) 
 
# according to slide, fitness value has been reduced to 1 dimension, so just use FitnessMax
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# Individual should be a list of binary values, i.e. a list of 0s and 1s
creator.create("Individual", list, fitness=creator.FitnessMax)


> Define the evaluate function for **FilterGA** and **WrapperGA**.
>
> Inspired by the slide.

In [ ]:
def evaluateFilterGA(ds:Sonar,individual:creator.Individual): 
    
    pass

def evaluateWrapperGA(ds:Sonar,individual:creator.Individual): 
    pass
